# Demo Logistic Regression

## Data Preparation

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import importlib
import sys
sys.path.append('..')  
from utils.ROCanalysis import calculate_roc_metrics

data = load_breast_cancer()
n_features = 4
X = data.data[:,:n_features]          # 4 numeric features
feature_names = data.feature_names[:n_features].tolist()
y = data.target        # 0 = malignant, 1 = benign

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Training
### Model training with sklearn

In [2]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(penalty=None,max_iter=10000)  # increase max_iter to ensure convergence
log_reg.fit(X_train_scaled, y_train)

y_pred = log_reg.predict(X_test_scaled)
accuracy = (y_test == y_pred).sum()/len(y_test)
print(f"Accuracy:{accuracy}")

proba = log_reg.predict_proba(X_test_scaled)[:,1]
roc_df = pd.DataFrame({
    'p': proba,
    'y': y_test,
    'w': np.ones(len(y_test))
})
print(calculate_roc_metrics(roc_df, 'p', 'y'))

all_params = np.concatenate([log_reg.coef_.ravel(), log_reg.intercept_])
coef_df = pd.DataFrame({
    "feature": list(feature_names)+['intercept'],
    "coefficient": all_params})
print(coef_df)

Accuracy:0.9298245614035088
{'auc': 98.347, 'gini': 96.693, 'ks': 85.913}
          feature  coefficient
0     mean radius    29.469177
1    mean texture    -0.986754
2  mean perimeter   -26.738366
3       mean area    -8.509443
4       intercept    -0.222692


### Model training with sklearn

Features of all samples $X\in \mathbb{R}^{n\times (K+1)}$ including the all one column, labels $\mathbf{y}\in {\mathbb{R}^n}$. The model output of all samples is

$$
\mathbf{p} = f(X;\mathbf{\beta}).
$$

The loss is

$$
l(\mathbf{\beta}) = -\frac{1}{n}\sum_{i=1}^n \left[y_i\ln p_i + (1-y_i)\ln(1-p_i)\right].
$$

whose gradient is 

$$
\frac{\partial}{\partial \mathbf{\beta}}l(\mathbf{\beta}) = \frac{1}{n} X^T(\mathbf{p}-\mathbf{y}).
$$


In python, `beta` is a (K+1,) np.array, `X` is (n,k) np.array.  Adding a all-one column `X` to have `X_new`, (n,k+1) np.array as

`X_new = np.hstack([X, np.ones((X.shape[0], 1))])`

The probabity of all sample `p` (n,) np.array is calculated as

`p = expit((X_new*self.beta).sum(axis=-1))`

loss is calculated as

`
loss = -(y * np.log(p) + (1 - y) * np.log(1-p)).mean()
`

The gradient is calculated as

`
grad = X_new.T @ (p - y)/n
`

Model is trained as

`
logreg.beta -= lr * (X_new.T @ (p - y)/n)
`

In [3]:
import numpy as np
from scipy.special import expit
class LogisticReg:
    def __init__(self, dim, epsilon=0.00000001):
        self.beta = np.random.rand(dim+1)
        print(self.beta.shape)
        self.epsilon = epsilon

    def _with_intercept(self, X):
        return np.hstack([X, np.ones((X.shape[0], 1))])

    def __call__(self, X):
        X_new = self._with_intercept(X)
        p = expit((X_new*self.beta).sum(axis=1))
        return p

    def loss(self, X, y):
        eps = self.epsilon
        p = np.clip(self(X), eps, 1-eps)
        loss = -(y * np.log(p) + (1 - y) * np.log(1-p)).mean()
        return loss

    def loss_grad(self, X, y):
        eps = self.epsilon
        n = X.shape[0]
        p = self(X)
        X_new = self._with_intercept(X)
        grad = X_new.T @ (p - y)/n # no clipping needed for bacprop
        return grad

logreg = LogisticReg(X_train.shape[1])
print(X_train_scaled.shape)
lr = 0.2
for i in range(500000):
    logreg.beta -= lr*logreg.loss_grad(X_train_scaled,y_train)
    if (i+1)%50000==0:
        print(f'loss = {logreg.loss(X_train_scaled,y_train)}')
        print(logreg.beta)

(5,)
(455, 4)
loss = 0.19915444466647877
[ 10.73267654  -0.93639379 -16.80290975   2.00618059   0.76455038]
loss = 0.19305564537551007
[ 16.76921509  -0.95260907 -20.89944454  -0.36044079   0.52951171]
loss = 0.19065515423510077
[ 20.61657655  -0.95879707 -22.80233852  -2.64223077   0.31612982]
loss = 0.18952695314591214
[ 23.23463865  -0.96410799 -23.96848997  -4.35105745   0.15841819]
loss = 0.18897535960951986
[ 25.05998528  -0.96921722 -24.77352482  -5.56229411   0.04707063]
loss = 0.188701339828801
[ 26.34610846  -0.9737368  -25.35149778  -6.41010947  -0.03078427]
loss = 0.1885638951126859
[ 27.2573195   -0.97745373 -25.7702799   -7.00378799  -0.08529471]
loss = 0.1884944947465408
[ 27.90510107  -0.98036848 -26.07370514  -7.42118271  -0.12362479]
loss = 0.18845928545561175
[ 28.36666429  -0.98258622 -26.29306743  -7.71592904  -0.15069719]
loss = 0.18844136137638656
[ 28.6960716   -0.98424121 -26.45129967  -7.92484923  -0.16988984]
